# Úloha klasifikace textu

Jak jsme již zmínili, zaměříme se na jednoduchou úlohu klasifikace textu založenou na datasetu **AG_NEWS**, kde je cílem zařadit nadpisy zpráv do jedné ze 4 kategorií: Svět, Sport, Byznys a Věda/Technika.

## Dataset

Tento dataset je součástí modulu [`torchtext`](https://github.com/pytorch/text), takže k němu máme snadný přístup.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Zde `train_dataset` a `test_dataset` obsahují kolekce, které vracejí dvojice štítku (číslo třídy) a textu, například:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Takže, pojďme vytisknout prvních 10 nových titulků z našeho datasetu:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Protože datové sady jsou iterátory, pokud chceme data použít vícekrát, musíme je převést na seznam:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizace

Nyní musíme převést text na **čísla**, která mohou být reprezentována jako tensory. Pokud chceme reprezentaci na úrovni slov, musíme udělat dvě věci:
* použít **tokenizér** k rozdělení textu na **tokeny**
* vytvořit **slovník** těchto tokenů.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Pomocí slovníku můžeme snadno zakódovat náš tokenizovaný řetězec do sady čísel:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Reprezentace textu pomocí Bag of Words

Protože slova nesou význam, někdy můžeme pochopit smysl textu jen tím, že se podíváme na jednotlivá slova, bez ohledu na jejich pořadí ve větě. Například při klasifikaci zpráv slova jako *počasí*, *sníh* pravděpodobně naznačují *předpověď počasí*, zatímco slova jako *akcie*, *dolar* by se počítala jako *finanční zprávy*.

**Bag of Words** (BoW) je nejčastěji používaná tradiční vektorová reprezentace. Každé slovo je spojeno s indexem vektoru, prvek vektoru obsahuje počet výskytů daného slova v konkrétním dokumentu.

![Obrázek ukazující, jak je v paměti reprezentována vektorová reprezentace Bag of Words.](../../../../../translated_images/cs/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Na BoW můžete také nahlížet jako na součet všech vektorů zakódovaných metodou one-hot pro jednotlivá slova v textu.

Níže je příklad, jak vytvořit reprezentaci Bag of Words pomocí python knihovny Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

K výpočtu vektoru bag-of-words z vektorové reprezentace našeho datasetu AG_NEWS můžeme použít následující funkci:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Poznámka:** Zde používáme globální proměnnou `vocab_size` k určení výchozí velikosti slovníku. Protože velikost slovníku je často poměrně velká, můžeme omezit jeho velikost na nejčastější slova. Zkuste snížit hodnotu `vocab_size` a spustit níže uvedený kód, abyste viděli, jak to ovlivní přesnost. Měli byste očekávat určitý pokles přesnosti, ale ne dramatický, výměnou za vyšší výkon.


## Trénování klasifikátoru BoW

Nyní, když jsme se naučili, jak vytvořit reprezentaci Bag-of-Words pro náš text, pojďme na ní natrénovat klasifikátor. Nejprve musíme převést náš dataset pro trénování tak, aby všechny poziční vektorové reprezentace byly převedeny na reprezentaci bag-of-words. Toho lze dosáhnout předáním funkce `bowify` jako parametru `collate_fn` do standardního torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Nyní definujme jednoduchou klasifikační neuronovou síť, která obsahuje jednu lineární vrstvu. Velikost vstupního vektoru se rovná `vocab_size` a velikost výstupu odpovídá počtu tříd (4). Protože řešíme klasifikační úlohu, konečná aktivační funkce je `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nyní definujeme standardní trénovací smyčku PyTorch. Protože náš dataset je poměrně velký, pro účely výuky budeme trénovat pouze jednu epochu, a někdy dokonce méně než jednu epochu (parametr `epoch_size` nám umožňuje omezit trénování). Během trénování také budeme hlásit kumulovanou přesnost trénování; frekvence hlášení je určena pomocí parametru `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGramy, TriGramy a N-Gramy

Jedním z omezení přístupu "bag of words" je, že některá slova jsou součástí víceslovných výrazů. Například slovo 'hot dog' má úplně jiný význam než slova 'hot' a 'dog' v jiných kontextech. Pokud bychom slova 'hot' a 'dog' vždy reprezentovali stejnými vektory, mohlo by to zmást náš model.

Abychom tento problém vyřešili, často se používají **reprezentace N-gramů** v metodách klasifikace dokumentů, kde frekvence každého slova, dvojice slov nebo trojice slov představuje užitečný prvek pro trénování klasifikátorů. Například v reprezentaci bigramů přidáme do slovníku všechny dvojice slov, kromě původních slov.

Níže je příklad, jak vytvořit reprezentaci bigramů pomocí Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Hlavní nevýhodou přístupu N-gram je, že velikost slovníku začíná růst extrémně rychle. V praxi je potřeba kombinovat reprezentaci N-gram s některými technikami redukce dimenzionality, jako jsou *embeddingy*, o kterých budeme diskutovat v další jednotce.

Abychom mohli použít reprezentaci N-gram v našem datasetu **AG News**, musíme vytvořit speciální slovník ngramů:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Mohli bychom použít stejný kód jako výše k natrénování klasifikátoru, avšak to by bylo velmi neefektivní z hlediska paměti. V další části natrénujeme bigramový klasifikátor pomocí embeddingů.

> **Poznámka:** Můžete ponechat pouze ty ngramy, které se v textu vyskytují vícekrát, než je stanovený počet. Tím zajistíte, že méně časté bigramy budou vynechány, a výrazně se sníží dimenzionalita. K tomu nastavte parametr `min_freq` na vyšší hodnotu a sledujte, jak se mění délka slovníku.


## Term Frequency Inverse Document Frequency TF-IDF

V reprezentaci BoW jsou výskyty slov rovnoměrně váženy, bez ohledu na samotné slovo. Je však zřejmé, že častá slova, jako například *a*, *v* atd., jsou pro klasifikaci mnohem méně důležitá než specializované termíny. Ve skutečnosti jsou v většině úloh NLP některá slova relevantnější než jiná.

**TF-IDF** znamená **termínová frekvence–inverzní dokumentová frekvence**. Jedná se o variaci bag of words, kde místo binární hodnoty 0/1, která označuje výskyt slova v dokumentu, se používá hodnota s plovoucí desetinnou čárkou, která souvisí s frekvencí výskytu slova v korpusu.

Formálně je váha $w_{ij}$ slova $i$ v dokumentu $j$ definována jako:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kde
* $tf_{ij}$ je počet výskytů $i$ v $j$, tj. hodnota BoW, kterou jsme viděli dříve
* $N$ je počet dokumentů v kolekci
* $df_i$ je počet dokumentů obsahujících slovo $i$ v celé kolekci

Hodnota TF-IDF $w_{ij}$ roste úměrně počtu výskytů slova v dokumentu a je upravena podle počtu dokumentů v korpusu, které obsahují dané slovo, což pomáhá zohlednit skutečnost, že některá slova se objevují častěji než jiná. Například pokud se slovo objeví *v každém* dokumentu v kolekci, $df_i=N$, a $w_{ij}=0$, a tyto termíny by byly zcela ignorovány.

TF-IDF vektorizaci textu můžete snadno vytvořit pomocí Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Závěr

I když TF-IDF reprezentace poskytují váhování frekvence různým slovům, nejsou schopny reprezentovat význam nebo pořadí. Jak řekl slavný lingvista J. R. Firth v roce 1935: „Úplný význam slova je vždy kontextový a žádné studium významu mimo kontext nelze brát vážně.“ Později v tomto kurzu se naučíme, jak zachytit kontextové informace z textu pomocí jazykového modelování.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
